# Clustering Crypto

In [96]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [2]:
# Load the crypto_data.csv dataset.
file_path = "./Resources/crypto_data.csv"
df = pd.read_csv(file_path, index_col=0)

df.head(10)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1252 entries, 42 to PUNK
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1252 non-null   object 
 1   Algorithm        1252 non-null   object 
 2   IsTrading        1252 non-null   bool   
 3   ProofType        1252 non-null   object 
 4   TotalCoinsMined  744 non-null    float64
 5   TotalCoinSupply  1252 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 59.9+ KB


In [4]:
# Keep all the cryptocurrencies that are being traded; IsTrading ==True.
df1 = df.loc[df['IsTrading'] == True]
df1.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1144 entries, 42 to XBC
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 54.7+ KB


In [6]:
# Based on info and sort below like all rows have a working Algorithm
# Check if any "Algorthm" values is NaN, blank or " ";
df0 = df1.sort_values(by='Algorithm', ascending=False)
df0.tail()

# Keep all the cryptocurrencies that have a working algorithm: drop NaN values; filtering Algorithm.
df2 = df1.dropna(axis=0, subset=['Algorithm'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1144 entries, 42 to XBC
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 54.7+ KB


In [7]:
# Remove the "IsTrading" column. 
df3 = df2.drop(['IsTrading'], axis=1)
df3

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [8]:
# Remove rows that have at least 1 null value.
df4 = df3.dropna()
df4

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [9]:
# Keep the rows where coins are mined.
df5 = df4.loc[df4['TotalCoinsMined'] > 0]
df5

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [10]:
# Create a new DataFrame that holds only the cryptocurrencies names.
cc_names_df = df5[["CoinName"]]
cc_names_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [11]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = df5.drop(['CoinName'], axis=1)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [12]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=[
                        'Algorithm',
                        'ProofType'])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [14]:
# Using PCA to reduce dimension to three principal components.
# Initialize PCA model
pca = PCA(n_components=3)

In [15]:
# Apply dimensionality reduction on the scaled dataset
# Get 3 principal components for the scaled crypto dataset.
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.33419736,  1.00933848, -0.54991606],
       [-0.31753804,  1.0094904 , -0.55038917],
       [ 2.30898759,  1.68053468, -0.68127406],
       ...,
       [ 0.33620004, -2.26208651,  0.36934082],
       [-0.18717655, -2.26093323,  0.52239478],
       [-0.28612612,  0.82290857, -0.26073507]])

In [16]:
index_values = (X.index.tolist())
index_values

['42',
 '404',
 '1337',
 'BTC',
 'ETH',
 'LTC',
 'DASH',
 'XMR',
 'ETC',
 'ZEC',
 'BTS',
 'DGB',
 'BTCD',
 'XPY',
 'PRC',
 'KOBO',
 'SPR',
 'ARG',
 'AUR',
 'BLU',
 'XMY',
 'MOON',
 'ZET',
 'SXC',
 'QTL',
 'ENRG',
 'QRK',
 'RIC',
 'DGC',
 'BTB',
 'CAT',
 'CBX',
 'CCN',
 'CRYPT',
 'CSC',
 'DMD',
 'XVG',
 'DVC',
 'EAC',
 'EFL',
 'EMC2',
 'EMD',
 'EXCL',
 'FLT',
 'FRK',
 'FTC',
 'GDC',
 'GLC',
 'GLD',
 'HBN',
 'HYP',
 'IFC',
 'IOC',
 'IXC',
 'KGC',
 'LKY',
 'LTB',
 'MAX',
 'MEC',
 'MED',
 'MINT',
 'MINC',
 'MZC',
 'NAUT',
 'NAV',
 'NOBL',
 'NMC',
 'NYAN',
 'OPAL',
 'ORB',
 'POT',
 'PXC',
 'RDD',
 'RPC',
 'SBC',
 'SMC',
 'SUPER',
 'SYNC',
 'SYS',
 'TES',
 'TGC',
 'TIT',
 'TOR',
 'TRC',
 'UNB',
 'UNO',
 'URO',
 'USDE',
 'UTC',
 'VIA',
 'VRC',
 'VTC',
 'WDC',
 'XC',
 'XCR',
 'XJO',
 'XST',
 'ZCC',
 'BCN',
 'XDN',
 'BURST',
 'SJCX',
 'MONA',
 'NTRN',
 'FAIR',
 'NLG',
 'RBY',
 'PTC',
 'KORE',
 'WBB',
 'NOTE',
 'FLO',
 '8BIT',
 'STV',
 'ABY',
 'FLDC',
 'U',
 'UIS',
 'CYP',
 'OMC',
 'VTR',
 'GRE'

In [81]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data = X_pca, columns=["PC 1", "PC 2", "PC 3"], index = index_values)

pcs_df

,PC 1,PC 2,PC 3
42,-0.334197,1.009338,-0.549916
404,-0.317538,1.009490,-0.550389
1337,2.308988,1.680535,-0.681274
BTC,-0.144656,-1.264208,0.176064
ETH,-0.151072,-2.059972,0.356587
...,...,...,...
ZEPH,2.469821,0.862199,-0.204383
GAP,-0.332242,1.009226,-0.549948
BDX,0.336200,-2.262087,0.369341
ZEN,-0.187177,-2.260933,0.522395


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [18]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

# Plot the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [86]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

pcs_df["Class"] = model.labels_

[1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1 1 1 1 0
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1
 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0 0 1 0 0 1
 1 0 1 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0
 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0
 1 0 1 0 1 1 3 0 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1
 1 0 0 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1
 0 0 1 0 0 0 0 1 3 3 0 0 0 1 3 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0
 1 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0
 1 1 1 1 0 1 1 0 1 1 0 3 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1
 0 0 0 1 1 1 0 1 0 1 0 1 

In [87]:
pcs_df

,PC 1,PC 2,PC 3,Class
42,-0.334197,1.009338,-0.549916,1
404,-0.317538,1.009490,-0.550389,1
1337,2.308988,1.680535,-0.681274,1
BTC,-0.144656,-1.264208,0.176064,0
ETH,-0.151072,-2.059972,0.356587,0
...,...,...,...,...
ZEPH,2.469821,0.862199,-0.204383,1
GAP,-0.332242,1.009226,-0.549948,1
BDX,0.336200,-2.262087,0.369341,0
ZEN,-0.187177,-2.260933,0.522395,0


In [88]:
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [92]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.

clustered_df = crypto_df.join(pcs_df, how='inner')

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = clustered_df.join(cc_names_df, how='inner')


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.334197,1.009338,-0.549916,1,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.317538,1.009490,-0.550389,1,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.308988,1.680535,-0.681274,1,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144656,-1.264208,0.176064,0,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.151072,-2.059972,0.356587,0,Ethereum
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.469821,0.862199,-0.204383,1,ZEPHYR
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.332242,1.009226,-0.549948,1,Gapcoin
BDX,CryptoNight,PoW,9.802226e+08,1400222610,0.336200,-2.262087,0.369341,0,Beldex
ZEN,Equihash,PoW,7.296538e+06,21000000,-0.187177,-2.260933,0.522395,0,Horizen


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [66]:
# Creating a 3D-Scatter with the PCA data and the clusters
# Plot the 3D-scatter with x="PC 1", y="PC 2" and z="PC 3"

fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_name="CoinName", hover_data=["Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
)

fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [94]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinsMined', 
                                  'TotalCoinSupply', 'Class'], sortable=True, selectable=True)


:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Class]

In [39]:
# Print the total number of tradable cryptocurrencies.
print(f"The Total Number of tradable cryptocurrencies is {clustered_df['CoinName'].count()}.")
  

The Total Number of tradable cryptocurrencies is 532.


In [44]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# transform only TotalCoinSupply, TotalCoinsMined

cluster_df = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
X_minmax = MinMaxScaler().fit_transform(cluster_df)
X_minmax

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [93]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
index_values = (clustered_df.index.tolist())

plot_df = pd.DataFrame(
    data = X_minmax, columns=["TotalCoinSupply_scaled", "TotalCoinsMined_scaled"], index = index_values)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df = plot_df.join(cc_names_df, how='inner')

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
class_df = clustered_df['Class']
plot_df = plot_df.join(class_df, how='inner')

plot_df

,TotalCoinSupply_scaled,TotalCoinsMined_scaled,CoinName,Class
42,4.200000e-11,0.000000e+00,42 Coin,1
404,5.320000e-04,1.065855e-03,404Coin,1
1337,3.141593e-01,2.957551e-02,EliteCoin,1
BTC,2.100000e-05,1.810842e-05,Bitcoin,0
ETH,0.000000e+00,1.087731e-04,Ethereum,0
...,...,...,...,...
ZEPH,2.000000e-03,2.020225e-03,ZEPHYR,1
GAP,2.500000e-04,1.508199e-05,Gapcoin,1
BDX,1.400223e-03,9.901351e-04,Beldex,0
ZEN,2.100000e-05,7.370282e-06,Horizen,0


In [64]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined_scaled", y="TotalCoinSupply_scaled", by="Class",
                      xlabel="Total Cryptocurrency Coins Mined",
                      ylabel="Total Cryptocurrency Coin Supply",
                      )



:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined_scaled]   (TotalCoinSupply_scaled)

## Re-do Deliverable 2 for 5 Classes/Clusters instead of 4

In [99]:
# Use a K-means function again to reuse the K-means cluster.
def get_clusters(k, data):
   # Create a copy of the DataFrame
   data = data.copy()
    
   # Initialize the K-Means model
   model = KMeans(n_clusters=k, random_state=0)

   # Fit the model
   model.fit(data)

   # Predict clusters
   predictions = model.predict(data)

   # Create return DataFrame with predicted clusters
   data["Class"] = model.labels_

   return data




In [100]:
# get 5 clusters
new_clusters = get_clusters(5, pcs_df)
new_clusters


,PC 1,PC 2,PC 3,Class
42,-0.334197,1.009338,-0.549916,0
404,-0.317538,1.009490,-0.550389,0
1337,2.308988,1.680535,-0.681274,4
BTC,-0.144656,-1.264208,0.176064,1
ETH,-0.151072,-2.059972,0.356587,1
...,...,...,...,...
ZEPH,2.469821,0.862199,-0.204383,4
GAP,-0.332242,1.009226,-0.549948,0
BDX,0.336200,-2.262087,0.369341,1
ZEN,-0.187177,-2.260933,0.522395,1


In [101]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.

clustered_df2 = crypto_df.join(new_clusters, how='inner')


#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df2 = clustered_df2.join(cc_names_df, how='inner')


# Print the shape of the clustered_df
print(clustered_df2.shape)
clustered_df2.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.334197,1.009338,-0.549916,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.317538,1.009490,-0.550389,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.308988,1.680535,-0.681274,4,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144656,-1.264208,0.176064,1,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.151072,-2.059972,0.356587,1,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.160345,-1.119140,0.020051,1,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.401519,1.209404,-0.522331,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.149207,-2.176657,0.410942,1,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.149513,-2.060069,0.356563,1,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.187175,-2.260933,0.522395,1,ZCash


In [102]:
clustered_df2.sort_values(by='Class')
clustered_df2.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.334197,1.009338,-0.549916,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.317538,1.009490,-0.550389,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.308988,1.680535,-0.681274,4,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144656,-1.264208,0.176064,1,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.151072,-2.059972,0.356587,1,Ethereum


In [103]:
fig = px.scatter_3d(
    clustered_df2,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_name="CoinName", hover_data=["Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
)

fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [105]:
# Create a table with tradable cryptocurrencies.
clustered_df2.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinsMined', 
                                  'TotalCoinSupply', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Class]

In [106]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# transform only TotalCoinSupply, TotalCoinsMined

cluster_df2 = clustered_df2[['TotalCoinSupply', 'TotalCoinsMined']]
X_minmax2 = MinMaxScaler().fit_transform(cluster_df2)
X_minmax2

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [107]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
index_values = (clustered_df2.index.tolist())

plot_df2 = pd.DataFrame(
    data = X_minmax2, columns=["TotalCoinSupply_scaled", "TotalCoinsMined_scaled"], index = index_values)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df2 = plot_df2.join(cc_names_df, how='inner')

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
class_df2 = clustered_df2['Class']
plot_df2 = plot_df2.join(class_df2, how='inner')

plot_df2

,TotalCoinSupply_scaled,TotalCoinsMined_scaled,CoinName,Class
42,4.200000e-11,0.000000e+00,42 Coin,0
404,5.320000e-04,1.065855e-03,404Coin,0
1337,3.141593e-01,2.957551e-02,EliteCoin,4
BTC,2.100000e-05,1.810842e-05,Bitcoin,1
ETH,0.000000e+00,1.087731e-04,Ethereum,1
...,...,...,...,...
ZEPH,2.000000e-03,2.020225e-03,ZEPHYR,4
GAP,2.500000e-04,1.508199e-05,Gapcoin,0
BDX,1.400223e-03,9.901351e-04,Beldex,1
ZEN,2.100000e-05,7.370282e-06,Horizen,1


In [108]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df2.hvplot.scatter(x="TotalCoinsMined_scaled", y="TotalCoinSupply_scaled", by="Class",
                      xlabel="Total Cryptocurrency Coins Mined",
                      ylabel="Total Cryptocurrency Coin Supply",
                      )


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined_scaled]   (TotalCoinSupply_scaled)